In [102]:
from crewai import Agent, Crew, Task, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool, FileReadTool, PDFSearchTool
from dotenv import load_dotenv
import os
from PyPDF2 import PdfReader
import json
import re
from pydantic import BaseModel, ValidationError
from llama_functions import llm_parse

load_dotenv()

# =======================
# MODELOS Pydantic
# =======================
class Address(BaseModel):
    street: str
    city: str
    state: str
    country: str
    zip_code: str

class Contact(BaseModel):
    phone: str
    email: str
    website: str
    address: list[Address]

class About(BaseModel):
    name: str
    position: str
    contact: Contact
    professional_objective: str

class Experience(BaseModel):
    titulo: str
    years: str
    institution: str
    description: str
    actual_position: str
    start_date: str
    end_date: str

class Education(BaseModel):
    diploma: str
    years: str
    institution: str
    description: str
    current: str
    start_date: str
    end_date: str
    city: str
    degree: str

class AdditionalCourse(BaseModel):
    course: str
    institution: str
    notes: str
    years: str
    start_date: str
    end_date: str

class Curriculo(BaseModel):
    about: About
    experience: list[Experience]
    education: list[Education]
    skills: list[str]
    languages: list
    additional_courses: list[AdditionalCourse]

class CurriculoModel(BaseModel):
    curriculos: list[Curriculo]

# Exemplo de JSON esperado (para referência na tarefa do agente)
json_example = """
{
    "curriculos": [
        {
            "about": {
                "name": "Nome do dono do currículo",
                "position": "Profissão",
                "contact": {
                    "phone": "Telefone",
                    "email": "Email",
                    "website": "Website (caso tenha)",
                    "address": [
                        {
                            "street": "Logradouro",
                            "city": "Cidade",
                            "state": "Estado",
                            "country": "País",
                            "zip_code": "CEP"
                        }
                    ]
                },
                "professional_objective": "Objetivo profissional (caso tenha)"
            },
            "experience": [
                {
                    "titulo": "Nome do cargo",
                    "years": "Data de início - Data de término",
                    "institution": "Nome da empresa",
                    "description": "Descrição (caso tenha)",
                    "actual_position": "yes",
                    "start_date": "2023-01-01",
                    "end_date": "2023-06-30"
                }
            ],
            "education": [
                {
                    "diploma": "Nome do curso",
                    "years": "Data de início - Data de término",
                    "institution": "Nome da instituição",
                    "description": "Descrição (caso tenha)",
                    "current": "yes",
                    "start_date": "2023-01-01",
                    "end_date": "2023-06-30",
                    "city": "Guarulhos",
                    "degree": "Bacharelado or Técnico"
                }
            ],
            "skills": ["Habilidade 1", "Habilidade 2", "Habilidade 3"],
            "languages": [],
            "additional_courses": [
                {
                    "course": "Python",
                    "institution": "OneBitCode",
                    "notes": "",
                    "years": "2023",
                    "start_date": "2023-01-01",
                    "end_date": "2023-06-30"
                },
                {
                    "course": "VueJs",
                    "institution": "OneBitCode",
                    "notes": "",
                    "years": "2023",
                    "start_date": "2023-01-01",
                    "end_date": "2023-06-30"
                }
            ]
        }
    ]
}
"""

# =======================
# FUNÇÃO DE CONVERSÃO
# =======================
def convert_pdf_to_json(pdf_file_path: str):
    """
    Converte um arquivo PDF de currículo em JSON estruturado.
    O arquivo PDF deve ser passado como argumento 'pdf_file_path'.
    """
    API_KEY = os.getenv("OP_AI_KEY")
    llm = LLM(model="gpt-4o-mini", temperature=0.5, api_key=API_KEY)

    # Lê o conteúdo do PDF
    reader = PdfReader(pdf_file_path)
    texto = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            texto += page_text + "\n"

    # =======================
    # CONFIGURAÇÃO DO AGENTE
    # =======================
    Leitor = Agent(
        role="Leitor de curriculo",
        goal="""Seu objetivo é carregar os dados do currículo que serão passados para o JSON. 
Os dados precisam ser exatamente os mesmos que estão no PDF.""",
        backstory="Você é um leitor de dados com vasta experiência em extrair informações de currículos e estruturá-los em JSON.",
        verbose=True,
        llm=llm,
    )

    carregar_curriculo = Task(
        description=f"""
Os dados devem ser carregados do seguinte texto extraído do PDF:
{texto}

Aja como um especialista em análise de currículos para extrair os dados estruturados no formato JSON.
Sua tarefa é processar o texto, identificar as informações conforme a estrutura definida abaixo,
e retornar exclusivamente um objeto JSON conforme o exemplo fornecido, sem nenhum texto adicional.

Instruções:
- O JSON deve conter os campos: about, experience, education, skills, languages, additional_courses.
- Se alguma informação não estiver disponível, mantenha a chave com valor "" ou uma lista vazia.
- Não adicione informações não presentes no currículo.
- Não repita chaves no JSON.

Exemplo de JSON esperado:
{json_example}
""",
        agent=Leitor,
        expected_output="JSON",
        llm=llm,
        verbose=True
    )

    crew = Crew(
        name="Curriculo",
        description="Processamento de currículo em PDF",
        tasks=[carregar_curriculo],
        llm=llm,
        verbose=True,
        agents=[Leitor],
        expected_output="JSON",
        json_dict=True,
    )

    crew_output = crew.kickoff()
    json_formatted = crew_output.raw.strip()

    # Remove aspas extras e backticks (se existirem)
    if json_formatted.startswith('"') and json_formatted.endswith('"'):
        json_formatted = json_formatted[1:-1]
    json_formatted = json_formatted.replace("```", "")

    # Tenta carregar o JSON
    try:
        data = json.loads(json_formatted)
    except json.JSONDecodeError as e:
        print("Erro ao decodificar o JSON:", e)
        data = None

    # Validação com Pydantic
    if data is not None:
        try:
            curriculo_model = CurriculoModel(**data)
        except ValidationError as ve:
            print("Erro na validação do JSON com Pydantic:", ve)
            curriculo_model = None

        if curriculo_model:
            # Exemplo: iterando sobre os currículos extraídos
            for curriculo in curriculo_model.curriculos:
                about = curriculo.about
                print("Nome:", about.name)
                print("Cargo:", about.position)
                print("Objetivo Profissional:", about.professional_objective)
                print("\nExperiências:")
                for exp in curriculo.experience:
                    print("  Título:", exp.titulo)
                    print("  Instituição:", exp.institution)
                    print("  Descrição:", exp.description)
                    print("-" * 40)
        return curriculo_model
    return None

# =======================
# EXECUÇÃO
# =======================
if __name__ == '__main__':
    # Informe o caminho do arquivo PDF que o agente deverá ler:
    pdf_path = "Lucas-Cv.pdf"  # Substitua pelo caminho real do seu PDF
    convert_pdf_to_json(pdf_path)


# Agent: Leitor de curriculo
## Task: 
Os dados devem ser carregados do seguinte texto extraído do PDF:
10/09/2021 14:49 https://vagas.canalconecta.com.br/curriculo?student-code=7d8d4dcf-3f3e-4c4f-ac1a-105b904126ab&source=student
https://vagas.canalconecta.com.br/curriculo?student-code=7d8d4dcf-3f3e-4c4f-ac1a-105b904126ab&source=student 1/1Lucas Pereira Souza
PESSOAL
Sexo: Masculino
Nascimento: 29/10/2001
CONTATO
Avenida Jorge Amado, 1027
Aquarela Ímbui TO B APT 508
Imbuí
CEP: 41720-040
Salvador/BA
 lucasp.sdev@gmail.com
 (71) 991848898
IDIOMA
Inglês (Intermediário)EDUCAÇÃO
Sistemas de Informação
Centro Universitário Anhanguera de Campo Grande
De 01/2020 a 12/2023
COMPETÊNCIAS TÉCNICAS
angular;  back-end;  bootstrap;  css 3;  figma;  front-end;  git - controle de versão
- (avançado);  github;  html 5;  java basico;  javascript;  laravel;  linux debian -
ubuntu - linux mint;  nodejs;  php7;  reactjs;  scss;  styled components; 
typescript; 
HISTÓRICO PROFISSIONAL
Ocupação Analista Jú

dict

In [90]:
json_formatted = crew_output.raw


In [91]:
json_limpo = json_formatted.strip()

# Se o JSON estiver envolto em aspas extras, remova-as:
if json_limpo.startswith('"') and json_limpo.endswith('"'):
    json_limpo = json_limpo[1:-1]

# Remover os backticks (se aparecerem)
json_limpo = json_limpo.replace("```", "")

# Agora, converta a string limpa para um dicionário
try:
    data = json.loads(json_limpo)
except json.JSONDecodeError as e:
    print("Erro ao decodificar o JSON:", e)
    data = None

if data is not None:
    # Iterando sobre os currículos
    for curriculo in data['curriculos']:
        about = curriculo['about']
        print("Nome:", about['name'])
        print("Cargo:", about['position'])
        print("Objetivo Profissional:", about['professional_objective'])
        print("\nExperiências:")
        for exp in curriculo['experience']:
            print("  Título:", exp['titulo'])
            print("  Instituição:", exp['institution'])
            print("  Descrição:", exp['description'])
            print("-" * 40)

Nome: Jonatas Silva
Cargo: Fullstack Developer
Objetivo Profissional: Desenvolvedor de Software Fullstack  com vasta experiência no ecossistema JavaScript/TypeScript.

Experiências:
  Título: Software Engineer
  Instituição: Forja
  Descrição: Atuei na Forja com um modelo semelhante a uma software house, onde fui responsável por desenvolver e entregar soluções completas para diversos tipos de projetos, desde front-end até back-end.
----------------------------------------
  Título: Desenvolvedor Fullstack
  Instituição: Infinity Tec
  Descrição: Desenvolvimento e manutenção de sites e sistemas, colaborando com equipes multifuncionais para entregar soluções robustas finalizando projetos 100% para os clientes.
----------------------------------------
  Título: Desenvolvedor Fullstack
  Instituição: Next Level Agency
  Descrição: Contribuição na construção de sites, aplicações e sistemas em uma startup contribuindo com cerca de 30% das aplicações já produzidas.
---------------------------

In [92]:
print(data)

{'curriculos': [{'about': {'name': 'Jonatas Silva', 'position': 'Fullstack Developer', 'contact': {'phone': '', 'email': '', 'website': 'jonatas-silva-developer.vercel.app/', 'address': [{'street': 'Crato', 'city': 'Crato', 'state': 'Ceará', 'country': 'Brasil', 'zip_code': ''}]}, 'professional_objective': 'Desenvolvedor de Software Fullstack  com vasta experiência no ecossistema JavaScript/TypeScript.'}, 'experience': [{'titulo': 'Software Engineer', 'years': 'outubro de 2024 - Present', 'institution': 'Forja', 'description': 'Atuei na Forja com um modelo semelhante a uma software house, onde fui responsável por desenvolver e entregar soluções completas para diversos tipos de projetos, desde front-end até back-end.', 'actual_position': 'yes', 'start_date': '2024-10-01', 'end_date': '2024-12-31'}, {'titulo': 'Desenvolvedor Fullstack', 'years': 'abril de 2024 - Present', 'institution': 'Infinity Tec', 'description': 'Desenvolvimento e manutenção de sites e sistemas, colaborando com equi

In [84]:
type(json_data)

str

In [78]:
print(crew_output.raw)

{
    "curriculos": [
        {
            "about": {
                "name": "Jonatas Silva",
                "position": "Fullstack Developer",
                "contact": {
                    "phone": "",
                    "email": "",
                    "website": "jonatas-silva-developer.vercel.app/",
                    "address": [
                        {
                            "street": "Crato",
                            "city": "Crato",
                            "state": "Ceará",
                            "country": "Brasil",
                            "zip_code": ""
                        }
                    ]
                },
                "professional_objective": "Desenvolvedor de Software Fullstack  com vasta experiência no ecossistema JavaScript/TypeScript."
            },
            "experience": [
                {
                    "titulo": "Software Engineer",
                    "years": "outubro de 2024 - Present",
                    "

In [77]:
resultado = crew_output.raw

json_objects = re.findall(r'\{.*?\}', resultado, re.DOTALL)

for json_str in json_objects:

    try:
        data = json.loads(json_str)
        print(data)  # Processar o JSON válido aqui
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")

Erro ao decodificar JSON: Expecting ',' delimiter: line 18 column 26 (char 626)
{'titulo': 'Software Engineer', 'years': 'outubro de 2024 - Present', 'institution': 'Forja', 'description': 'Atuei na Forja com um modelo semelhante a uma software house, onde fui responsável por desenvolver e entregar soluções completas para diversos tipos de projetos, desde front-end até back-end.', 'actual_position': 'yes', 'start_date': '2024-10-01', 'end_date': '2024-12-31'}
{'titulo': 'Desenvolvedor Fullstack', 'years': 'abril de 2024 - Present', 'institution': 'Infinity Tec', 'description': 'Desenvolvimento e manutenção de sites e sistemas, colaborando com equipes multifuncionais para entregar soluções robustas finalizando projetos 100% para os clientes.', 'actual_position': 'yes', 'start_date': '2024-04-01', 'end_date': '2024-12-31'}
{'titulo': 'Desenvolvedor Fullstack', 'years': 'abril de 2024 - outubro de 2024', 'institution': 'Next Level Agency', 'description': 'Contribuição na construção de sit

In [40]:
crew_output

CrewOutput(raw='{\n    "curriculos": [\n        {\n            "about": {\n                "name": "Jonatas Silva",\n                "position": "Fullstack Developer",\n                "contact": {\n                    "phone": "",\n                    "email": "",\n                    "website": "jonatas-silva-developer.vercel.app/",\n                    "address": [\n                        {\n                            "street": "Crato",\n                            "city": "Crato",\n                            "state": "Ceará",\n                            "country": "Brasil",\n                            "zip_code": ""\n                        }\n                    ]\n                },\n                "professional_objective": "Desenvolvedor de Software Fullstack com vasta experiência no ecossistema JavaScript/TypeScript. Sempre buscando oportunidades para aprender e evoluir, acredito que o aprendizado contínuo é o fundamental para melhor desenvolver soluções."\n            },

In [ ]:
resultado

'{\n    "curriculos": [\n        {\n            "about": {\n                "name": "Jonatas Silva",\n                "position": "Fullstack Developer",\n                "contact": {\n                    "phone": "",\n                    "email": "",\n                    "website": "jonatas-silva-developer.vercel.app/",\n                    "address": [\n                        {\n                            "street": "Crato",\n                            "city": "Crato",\n                            "state": "Ceará",\n                            "country": "Brasil",\n                            "zip_code": ""\n                        }\n                    ]\n                },\n                "professional_objective": "Desenvolvedor de Software Fullstack com vasta experiência no ecossistema JavaScript/TypeScript. Sempre buscando oportunidades para aprender e evoluir, acredito que o aprendizado contínuo é o fundamental para melhor desenvolver soluções."\n            },\n            "

In [44]:
resultados_json = resultado.split('}\n{')

# Adicionando as chaves de abertura e fechamento para cada objeto
resultados_json = ['{' + item + '}' for item in resultados_json]

# Carregando os objetos JSON individualmente
for resultado in resultados_json:
    try:
        data = json.loads(resultado)
        # Aqui você pode processar os dados de cada JSON individualmente
        for curriculo in data["curriculos"]:
            print(f"Nome: {curriculo['about']['name']}")
            print(f"Cargo: {curriculo['about']['position']}")
            
            # Experiência
            print("Experiência:")
            for exp in curriculo['experience']:
                print(f"- {exp['titulo']} ({exp['years']})")
            print()
    except json.JSONDecodeError as e:
        print(f"Erro ao carregar o JSON: {e}")

Erro ao carregar o JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


In [27]:
type(json_objects)

list

In [14]:
resultado = crew_output.raw
match = re.search(r"\{.*\}", resultado, re.DOTALL)

if match:
    crew_json = json.loads(match.group())  # Converte para JSON
else:
    raise ValueError("JSON inválido ou não encontrado.")

In [16]:
print(match)

<re.Match object; span=(0, 7354), match='{\n    "curriculos": [\n        {\n            "a>
